In [1]:
from osgeo import gdal
import ogr
import matplotlib.pyplot as plt
import numpy as np
import fiona
import xgboost
import pandas as pd

# Set seed
np.random.seed(100)

# Read files 
trainX = np.load('/scratch/slums/bl-slums/gt/final-px-tr-3-Xa')
trainY = np.load('/scratch/slums/bl-slums/gt/final-px-tr-3-Ya')
testX = np.load('/scratch/slums/bl-slums/gt/final-px-te-3-Xa')
testY = np.load('/scratch/slums/bl-slums/gt/final-px-te-3-Ya')

trainY = trainY.ravel()
testY = testY.ravel()
print trainX.shape, trainY.shape, testX.shape, testY.shape

/afs/unity.ncsu.edu/users/k/kgadira/.local/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


(3547, 18) (3547,) (779, 18) (779,)


'\nnTestSlums = np.sum(testY==0)\n\n# print nTestSlums\n# Pick all other attributes in number equal to nTestSlums randomly\nclass0Indices = np.where(testY==0)\nclass1Indices = np.where(testY==1)\nclass2Indices = np.where(testY==2)\n\nprint len(class1Indices[0])\nindices1 = class1Indices[0][np.random.choice(len(class1Indices[0]), nTestSlums)]\nindices2 = class2Indices[0][np.random.choice(len(class2Indices[0]), nTestSlums)]\n\nclass0X = testX[class0Indices[0],:]\nclass1X = testX[indices1,:]\nclass2X = testX[indices2,:]\n\nprint class0X.shape, class1X.shape, class2X.shape\nclass0Y = testY[class0Indices]\nclass1Y = testY[indices1]\nclass2Y = testY[indices2]\n\ntestX = np.vstack((class0X, class1X, class2X))\ntestY = np.hstack((class0Y, class1Y, class2Y))\n\nprint testX.shape, testY.shape\n'

In [2]:
trainX = np.nan_to_num(trainX)
testX = np.nan_to_num(testX)

In [3]:
# Utility function to report best scores
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

In [4]:
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import pickle
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from matplotlib import pyplot
model = xgboost.XGBClassifier(nthread=16 ,objective='multi:softmax')
learning_rate = [0.2, 0.3, 0.5, 0.7, 0.9]
n_estimators = [1000,5000,6000, 7000, 10000]
param_grid = dict(learning_rate=learning_rate, n_estimators = n_estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=7)
grid_search = GridSearchCV(model, param_grid, scoring="neg_log_loss", n_jobs=-1, cv=kfold)
grid_result = grid_search.fit(trainX, trainY)
report(grid_result.cv_results_)


Model with rank: 1
Mean validation score: -0.494 (std: 0.039)
Parameters: {'n_estimators': 1000, 'learning_rate': 0.2}

Model with rank: 2
Mean validation score: -0.569 (std: 0.035)
Parameters: {'n_estimators': 1000, 'learning_rate': 0.3}

Model with rank: 3
Mean validation score: -0.639 (std: 0.044)
Parameters: {'n_estimators': 1000, 'learning_rate': 0.5}



In [2]:
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import pickle
#kfold = StratifiedKFold(n_splits = 10, random_state=7)
xgb = xgboost.XGBClassifier(max_depth=500, n_estimators=1000, nthread=8 , objective='multi:softmax', learning_rate = 0.2 )
#results = cross_val_score(xgb, trainX, trainY, cv=kfold)
#print(results)
xgb.fit(trainX,trainY)
result = xgb.predict(testX)
acc = accuracy_score(testY, result)
cm = confusion_matrix(testY, result)
cr = classification_report(testY,result, target_names=['Informal','Formal','Other'])
print 'Overall accuracy = {}\n'.format(acc)
#print 'Slum accuracy = {}\n'.format(cm[0,0]/np.sum(cm[0,:]))
print 'Confusion Matrix \n {}\n'.format(cm)
print 'Classification Report \n {}\n'.format(cr)

fname = 'VHR-xgboost-3-Cl-model.sav'
pickle.dump(xgb, open(fname, 'wb'))

Overall accuracy = 0.83055198973

Confusion Matrix 
 [[148  86   1]
 [ 38 164   0]
 [  1   6 335]]

Classification Report 
              precision    recall  f1-score   support

   Informal       0.79      0.63      0.70       235
     Formal       0.64      0.81      0.72       202
      Other       1.00      0.98      0.99       342

avg / total       0.84      0.83      0.83       779




In [15]:
# Read the image samples created before
from osgeo import gdal
import ogr, osr
import matplotlib.pyplot as plt
import numpy as np
import fiona

ind = 3

driver = gdal.GetDriverByName('GTiff')
#rasterFileName = "/home/gadiraju/data/bl-slums/raw-img/MUL_mosaic_415.tif" #path to raster
rasterFileName1 = "/scratch/slums/bl-slums/raw-img/PS_mosaic_extract_{}.tif".format(ind)
dataset1 = gdal.Open(rasterFileName1)
rasterFileName2 = "/scratch/slums/bl-slums/raw-img/PS_mosaic_NDBI_extract_{}.tif".format(ind)
dataset2 = gdal.Open(rasterFileName2)
rasterFileName3 = "/scratch/slums/bl-slums/raw-img/PAN_mosaic_simple_extract_{}.tif".format(ind)
dataset3 = gdal.Open(rasterFileName3)
rasterFileName4 = "/scratch/slums/bl-slums/raw-img/PAN_mosaic_edgeDensity_{}.tif".format(ind)
dataset4 = gdal.Open(rasterFileName4)
imggeotrans1 = dataset1.GetGeoTransform()
coordinates_list = []

bands1=[]
bands2=[]
bands3=[]
bands4=[]
data_all_bands = []
cols = dataset1.RasterXSize
rows = dataset1.RasterYSize

transform = dataset1.GetGeoTransform()

imggeotrans2 = dataset2.GetGeoTransform()
imggeotrans3 = dataset3.GetGeoTransform()


print '{} \n +++++++++++++++++++++++++++ \n {} \n {}'.format(imggeotrans1, imggeotrans2, imggeotrans3)

xOrigin = transform[0]
yOrigin = transform[3]
pixelWidth = transform[1]
pixelHeight = -transform[5]


print xOrigin, yOrigin, pixelWidth, pixelHeight

for i in range(8):
    bands1.append(dataset1.GetRasterBand(i+1)) 
    # data_all_bands.append(band.ReadAsArray(0,0,cols,rows).astype(np.float))
bands2.append(dataset2.GetRasterBand(1))
for i in range(8):
    bands3.append(dataset3.GetRasterBand(i+1)) 
bands4.append(dataset4.GetRasterBand(1))

curr_img = np.zeros((rows,cols,18))-1
for k in range(8):
    data = bands1[k].ReadAsArray(0,0,cols,rows).astype(np.float)
    curr_img[:,:,k] = data
for k in range(8):
    data = bands3[k].ReadAsArray(0,0,cols,rows).astype(np.float)
    curr_img[:,:,8+k] = data 
data = bands2[0].ReadAsArray(0,0,cols,rows).astype(np.float)
curr_img[:,:,16] = data
data = bands4[0].ReadAsArray(0,0,cols,rows).astype(np.float)
curr_img[:,:,17] = data
print rows, cols
result_raster = np.zeros((rows, cols))-1
for i in range(curr_img.shape[0]):
    curr_line = curr_img[i,:,:]
    result_raster[i,:] = xgb.predict(curr_line)


def write_image(imgrows, imgcols, imgbands, imggeotrans,edges, opath):
    driver = gdal.GetDriverByName('GTiff')
    outRaster = driver.Create(opath, imgrows, imgcols, imgbands, gdal.GDT_Float32)
    outRaster.SetGeoTransform(imggeotrans)
    outband = outRaster.GetRasterBand(1)
    outband.WriteArray(edges)
    outRasterSRS = osr.SpatialReference()
    outRasterSRS.ImportFromEPSG(32643)
    outRaster.SetProjection(outRasterSRS.ExportToWkt())
    outband.FlushCache()

print np.sum(result_raster==0), np.sum(result_raster==1), np.sum(result_raster==2)

write_image(cols, rows, 1, transform,result_raster, '/scratch/slums/bl-slums/clf-img/section-{}'.format(ind))

(782148.3121834793, 0.499284950117, 0.0, 1428678.0102606558, 0.0, -0.499231422086) 
 +++++++++++++++++++++++++++ 
 (782148.3121834793, 0.499284950117, 0.0, 1428678.0102606558, 0.0, -0.499231422086) 
 (782148.3121834793, 0.499284950117, 0.0, 1428678.0102606558, 0.0, -0.499231422086)
782148.312183 1428678.01026 0.499284950117 0.499231422086
482 448
115549 46071 54316


In [10]:
from sklearn.ensemble import RandomForestClassifier

# Utility function to report best scores
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")


rf = RandomForestClassifier(n_estimators = 500)
param_grid = {"max_depth": [3, None],
              "min_samples_split": [2, 3],
              "min_samples_leaf": [1, 3],
              "n_estimators": [500, 1000],
              "bootstrap": [True, False],
              "criterion": ["gini", "entropy"]}
grid_search = GridSearchCV(rf, param_grid = param_grid, scoring="neg_log_loss", n_jobs=-1, cv=kfold)
grid_result = grid_search.fit(trainX, trainY)
report(grid_result.cv_results_)



Model with rank: 1
Mean validation score: -0.342 (std: 0.011)
Parameters: {'bootstrap': False, 'min_samples_leaf': 1, 'n_estimators': 1000, 'criterion': 'entropy', 'min_samples_split': 3, 'max_depth': None}

Model with rank: 2
Mean validation score: -0.343 (std: 0.012)
Parameters: {'bootstrap': False, 'min_samples_leaf': 1, 'n_estimators': 500, 'criterion': 'entropy', 'min_samples_split': 2, 'max_depth': None}

Model with rank: 3
Mean validation score: -0.343 (std: 0.011)
Parameters: {'bootstrap': False, 'min_samples_leaf': 1, 'n_estimators': 500, 'criterion': 'entropy', 'min_samples_split': 3, 'max_depth': None}



In [11]:
rf = RandomForestClassifier(n_estimators = 1000, bootstrap = False, min_samples_leaf = 1, min_samples_split = 3, max_depth = None)
rf.fit(trainX,trainY)
result = rf.predict(testX)
acc = accuracy_score(testY, result)
cm = confusion_matrix(testY, result)
cr = classification_report(testY,result, target_names=['Informal','Formal','Other'])
print 'Overall accuracy = {}\n'.format(acc)
#print 'Slum accuracy = {}\n'.format(cm[0,0]/np.sum(cm[0,:]))
print 'Confusion Matrix \n {}\n'.format(cm)
print 'Classification Report \n {}\n'.format(cr)
fname = 'VHR-rf-2-Cl-model.sav'
pickle.dump(rf, open(fname, 'wb'))


Overall accuracy = 0.829268292683

Confusion Matrix 
 [[148  86   1]
 [ 35 166   1]
 [  1   9 332]]

Classification Report 
              precision    recall  f1-score   support

   Informal       0.80      0.63      0.71       235
     Formal       0.64      0.82      0.72       202
      Other       0.99      0.97      0.98       342

avg / total       0.84      0.83      0.83       779




In [12]:
from sklearn.naive_bayes import GaussianNB

nb = GaussianNB()
nb.fit(trainX,trainY)
result = nb.predict(testX)
acc = accuracy_score(testY, result)
cm = confusion_matrix(testY, result)
cr = classification_report(testY,result, target_names=['Informal','Formal','BG'])
print 'Overall accuracy = {}\n'.format(acc)
print 'Confusion Matrix \n {}\n'.format(cm)
print 'Classification Report \n {}\n'.format(cr)

fname = 'VHR-gnb-3-Cl-model.sav'
pickle.dump(nb, open(fname, 'wb'))

Overall accuracy = 0.771501925546

Confusion Matrix 
 [[208  24   3]
 [127  73   2]
 [  3  19 320]]

Classification Report 
              precision    recall  f1-score   support

   Informal       0.62      0.89      0.73       235
     Formal       0.63      0.36      0.46       202
         BG       0.98      0.94      0.96       342

avg / total       0.78      0.77      0.76       779




In [13]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=100)
dt.fit(trainX,trainY)
result = dt.predict(testX)
acc = accuracy_score(testY, result)
cm = confusion_matrix(testY, result)
cr = classification_report(testY,result, target_names=['Informal','Formal','BG'])
print 'Overall accuracy = {}\n'.format(acc)
#print 'Slum accuracy = {}\n'.format(cm[0,0]/np.sum(cm[0,:]))
print 'Confusion Matrix \n {}\n'.format(cm)
print 'Classification Report \n {}\n'.format(cr)

fname = 'VHR-dt-3-Cl-model.sav'
pickle.dump(dt, open(fname, 'wb'))

Overall accuracy = 0.795892169448

Confusion Matrix 
 [[145  89   1]
 [ 51 150   1]
 [  4  13 325]]

Classification Report 
              precision    recall  f1-score   support

   Informal       0.72      0.62      0.67       235
     Formal       0.60      0.74      0.66       202
         BG       0.99      0.95      0.97       342

avg / total       0.81      0.80      0.80       779




In [15]:
from sklearn.neighbors import KNeighborsClassifier
cv = StratifiedKFold(n_splits = 10, random_state =7 )
knn = KNeighborsClassifier()
n_neighbors = list(np.arange(3,11,1))
#print n_neighs
params_grid = dict(n_neighbors= n_neighbors)
knn_grid_search = GridSearchCV(estimator = knn, n_jobs = -1, param_grid = params_grid)
knn_grid_result = knn_grid_search.fit(trainX, trainY)
report(knn_grid_result.cv_results_)

knn = KNeighborsClassifier(n_neighbors = 10)
knn.fit(trainX, trainY)
result= knn.predict(testX)
acc = accuracy_score(testY, result)
cm = confusion_matrix(testY, result)
cr = classification_report(testY,result, target_names=['Informal','Formal','Other'])
print 'Overall accuracy = {}\n'.format(acc)
#print 'Slum accuracy = {}\n'.format(cm[0,0]/np.sum(cm[0,:]))
print 'Confusion Matrix \n {}\n'.format(cm)
print 'Classification Report \n {}\n'.format(cr)

fname = 'VHR-knn-2-Cl-model.sav'
pickle.dump(knn, open(fname, 'wb'))


Model with rank: 1
Mean validation score: 0.651 (std: 0.083)
Parameters: {'n_neighbors': 10}

Model with rank: 2
Mean validation score: 0.650 (std: 0.082)
Parameters: {'n_neighbors': 9}

Model with rank: 3
Mean validation score: 0.650 (std: 0.079)
Parameters: {'n_neighbors': 5}

Model with rank: 3
Mean validation score: 0.650 (std: 0.088)
Parameters: {'n_neighbors': 6}

Overall accuracy = 0.739409499358

Confusion Matrix 
 [[152  76   7]
 [102  90  10]
 [  1   7 334]]

Classification Report 
              precision    recall  f1-score   support

   Informal       0.60      0.65      0.62       235
     Formal       0.52      0.45      0.48       202
      Other       0.95      0.98      0.96       342

avg / total       0.73      0.74      0.73       779




In [16]:
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier()
params_grid={
'learning_rate': ["constant", "invscaling", "adaptive"],
'hidden_layer_sizes': [(100,100,100,100), (100,100,100,100,100), (100,100,100,100,100,100,100,100)],
'alpha': [0.0001, 0.00001, 0.01],
'activation': ["logistic", "relu", "tanh"]
}

mlp_grid_search = GridSearchCV(estimator=mlp,param_grid=params_grid,n_jobs=-1,cv=kfold)
mlp_grid_result = mlp_grid_search.fit(trainX, trainY)
report(mlp_grid_result.cv_results_)


Model with rank: 1
Mean validation score: 0.743 (std: 0.026)
Parameters: {'alpha': 0.01, 'activation': 'relu', 'learning_rate': 'constant', 'hidden_layer_sizes': (100, 100, 100, 100, 100, 100, 100, 100)}

Model with rank: 2
Mean validation score: 0.743 (std: 0.018)
Parameters: {'alpha': 1e-05, 'activation': 'relu', 'learning_rate': 'invscaling', 'hidden_layer_sizes': (100, 100, 100, 100, 100, 100, 100, 100)}

Model with rank: 3
Mean validation score: 0.739 (std: 0.022)
Parameters: {'alpha': 0.0001, 'activation': 'relu', 'learning_rate': 'adaptive', 'hidden_layer_sizes': (100, 100, 100, 100, 100, 100, 100, 100)}



In [18]:
mlp = MLPClassifier(activation='relu', learning_rate = 'constant',hidden_layer_sizes=(100, 100, 100, 100,100,100,100,100), alpha = 0.01)
mlp.fit(trainX, trainY)
result = mlp.predict(testX)
acc = accuracy_score(testY, result)
cm = confusion_matrix(testY, result)
cr = classification_report(testY,result, target_names=['Informal','Formal','Other'])
print 'Overall accuracy = {}\n'.format(acc)
#print 'Slum accuracy = {}\n'.format(cm[0,0]/np.sum(cm[0,:]))
print 'Confusion Matrix \n {}\n'.format(cm)
print 'Classification Report \n {}\n'.format(cr)

fname = 'VHR-mlp-3-Cl-model.sav'
pickle.dump(mlp, open(fname, 'wb'))

Overall accuracy = 0.767650834403

Confusion Matrix 
 [[ 88 138   9]
 [ 24 170   8]
 [  0   2 340]]

Classification Report 
              precision    recall  f1-score   support

   Informal       0.79      0.37      0.51       235
     Formal       0.55      0.84      0.66       202
      Other       0.95      0.99      0.97       342

avg / total       0.80      0.77      0.75       779




In [20]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import AdaBoostClassifier
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=7)
adb = AdaBoostClassifier()
params_grid = dict(n_estimators=[50,100,500,1000,5000], learning_rate=[0.01, 0.007, 0.0001, 0.1, 0.0007])
adb_grid_search = GridSearchCV(estimator=adb, param_grid = params_grid, cv=kfold)
adb_search_result = adb_grid_search.fit(trainX, trainY)
report(adb_search_result.cv_results_)

Model with rank: 1
Mean validation score: 0.749 (std: 0.017)
Parameters: {'n_estimators': 1000, 'learning_rate': 0.1}

Model with rank: 2
Mean validation score: 0.747 (std: 0.020)
Parameters: {'n_estimators': 500, 'learning_rate': 0.1}

Model with rank: 3
Mean validation score: 0.746 (std: 0.021)
Parameters: {'n_estimators': 5000, 'learning_rate': 0.01}



In [21]:
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import pickle
adb = AdaBoostClassifier(n_estimators=1000, learning_rate=0.1)
adb.fit(trainX, trainY)
result = adb.predict(testX)
acc = accuracy_score(testY, result)
cm = confusion_matrix(testY, result)
cr = classification_report(testY,result, target_names=['Informal','Formal','Other'])
print 'Overall accuracy = {}\n'.format(acc)
#print 'Slum accuracy = {}\n'.format(cm[0,0]/np.sum(cm[0,:]))
print 'Confusion Matrix \n {}\n'.format(cm)
print 'Classification Report \n {}\n'.format(cr)

fname = 'VHR-adaboost-3-Cl-model.sav'
pickle.dump(adb, open(fname, 'wb'))

Overall accuracy = 0.770218228498

Confusion Matrix 
 [[175  59   1]
 [107  95   0]
 [  0  12 330]]

Classification Report 
              precision    recall  f1-score   support

   Informal       0.62      0.74      0.68       235
     Formal       0.57      0.47      0.52       202
      Other       1.00      0.96      0.98       342

avg / total       0.77      0.77      0.77       779


